In [2]:
import torch
import os
import numpy as np
import NeuralTSNE as ntsne
import matplotlib.pyplot as plt

from torch.utils.data import Subset, ConcatDataset
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer

In [ ]:
# Load the dataset and split it into training and testing sets
try:
  train, test = ntsne.DatasetLoader.get_datasets.prepare_dataset("colvar")
except FileNotFoundError:
  print("Dataset not found. Please download the dataset first.")
  exit(1)
  
val = Subset(train, range(0, len(test)))
train = Subset(train, range(len(test), len(train)))

Dataset not found. Please download the dataset first.


In [ ]:
features = train.dataset[0][0].size(0)
reduced_features = 2
multipliers = [0.75, 0.75, 0.75]
basic_model = ntsne.TSNE.NeuralNetwork.NeuralNetwork(
    features, reduced_features, multipliers
)

In [ ]:
param_tsne = ntsne.TSNE.ParametricTSNE.ParametricTSNE(
    loss_fn="kl_divergence",
    n_components=reduced_features,
    perplexity=30,
    batch_size=1000,
    early_exaggeration_epochs=0,
    early_exaggeration_value=12,
    max_iterations=500,
    features=features,
    multipliers=multipliers,
)

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=10)
trainer = Trainer(
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    log_every_n_steps=1,
    max_epochs=param_tsne.max_iterations,
    callbacks=[early_stopping],
)

In [ ]:
reducer = ntsne.TSNE.Modules.DimensionalityReduction(param_tsne, False, lr=1e-3)

traind, testd = param_tsne.create_dataloaders(train, test)
vald, _ = param_tsne.create_dataloaders(val, None)

all_data = ConcatDataset([train, val, test])
all_loader, _ = param_tsne.create_dataloaders(all_data, None)

In [ ]:
torch.set_float32_matmul_precision("high")

if os.path.exists("colvar_model.pt"):
    param_tsne.read_model("colvar_model.pt")
else:
    trainer.fit(reducer, traind, [vald])
    param_tsne.save_model("colvar_model.pt")

In [ ]:
Y = trainer.predict(reducer, testd)
Y = np.concatenate(Y, axis=0)

header = ','.join(["x1", "x2"])
np.savetxt("colvar_2d.csv", Y, header=header, delimiter=",")

x = Y[:, 0]
y = Y[:, 1]

In [ ]:
plt.scatter(x, y)
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Scatter Plot of 2D Vectors")

plt.show()